In [30]:
import pickle
import json
import pandas as pd
import numpy as np
import xgboost

In [31]:
model = xgboost.XGBClassifier()
# model.load_model('../models/nsql_model_v2.json')
model.load_model('./nsql_model_v2.json')
# model.save_model('./nsql_model_v2.json')

In [32]:
def load_models():
    file_name = "./nsql_model_v2.json"
    model = xgboost.XGBClassifier()
    model.load_model(file_name)

    return model

def convert_nulls_to_one_format(df:pd.DataFrame):
    for col in df.columns:
        idx = df.index[df[col].isnull()].tolist()
        idx.extend(df.index[df[col].isna()].tolist())
        idx.extend(df.index[df[col] == ''].tolist())
        idx.extend(df.index[df[col] == '[]'].tolist())
        idx = list(set(idx))
        df.loc[idx, col] = None
    return df

def impute_nulls_zeros(df:pd.DataFrame):
    ################## APPLICATIONS ##################
    df['estimated_monthly_revenue'] = df['estimated_monthly_revenue'].fillna('$1k +')
    df['incoming_ach_payments'] = df['incoming_ach_payments'].fillna('<$1k')
    df['outgoing_ach_and_checks'] = df['outgoing_ach_and_checks'].fillna('<$1k')
    df['check_deposit_amount'] = df['check_deposit_amount'].fillna('<$1k')
    df['outgoing_wire_transfers'] = df['outgoing_wire_transfers'].fillna('$0')
    df['incoming_wire_transfer'] = df['incoming_wire_transfer'].fillna('$0')
    df['business_type'] = df['business_type'].fillna('llc')
    df['email_domain'] = df['email_domain'].fillna('gmail.com')
    df['current_bank'] = df['current_bank'].fillna('novo-is-first')
    df['industry_category_name'] = df['industry_category_name'].fillna('retail trade')
    ################## ALLOY ##################
    df['iovation_device_type'] = df['iovation_device_type'].fillna('iphone')
    df['iovation_device_timezone'] = df['iovation_device_timezone'].fillna('300')
    df['carrier'] = df['carrier'].fillna('t-mobile usa')
    df['socure_sigma'] = df['socure_sigma'].fillna(0.557)
    df['socure_phonerisk'] = df['socure_phonerisk'].fillna(0.408)
    df['socure_emailrisk'] = df['socure_emailrisk'].fillna(0.879)
    df['socure_reason_code'] = df['socure_reason_code'].fillna("""[\n  "I610",\n  "R566",\n  "I711",\n  "I632",\n  "I705",\n  "I625",\n  "I611",\n  "I614",\n  "I636",\n  "R561",\n  "I630",\n  "I708",\n  "I618",\n  "I555",\n  "I707",\n  "I602"\n]""")
    df['socure_phonerisk_reason_code'] = df['socure_phonerisk_reason_code'].fillna("""[\n  "I610",\n  "I632",\n  "I620",\n  "I625",\n  "I611",\n  "I614",\n  "I636",\n  "I630",\n  "I618",\n  "I602"\n]""")
    df['socure_emailrisk_reason_code'] = df['socure_emailrisk_reason_code'].fillna("""[\n  "I520",\n  "I553",\n  "I556",\n  "I555"\n]""")
    ################## SEGMENT ##################
    df['screen_width_mean'] = df['screen_width_mean'].fillna(1092.0)
    df['screen_width_mean'] = np.where(df['screen_width_mean']==0, 1092.0, df['screen_width_mean'])
    df['screen_height_mean'] = df['screen_height_mean'].fillna(751.0)
    df['screen_height_mean'] = np.where(df['screen_height_mean']==0, 751.0, df['screen_height_mean'])
    
    return df

def derive_variables(df:pd.DataFrame):
   
    ################## APPLICATIONS ##################
    # estimated business numbers
    estimated_cols = ['estimated_monthly_revenue', 'incoming_ach_payments', 'outgoing_ach_and_checks', 
                      'check_deposit_amount', 'outgoing_wire_transfers', 'incoming_wire_transfer']

    # grouping all responses into 5K+ and 5K-
    for col in estimated_cols:
        df[col] = df[col].str.lower()
        df[col] = np.where(df[col].isin(['$5k +', '$50k +']), 1, 0)

    # business type
    df['business_type'] = df['business_type'].str.lower()    
    df['business_group'] = np.where(df['business_type'] == 'sole_proprietorship', 0, 1)

    # current bank
    df['current_bank'] = df['current_bank'].str.lower()    
    hdb_group = ['bluevine', 'other-national-bank', 'td-ank', 'chase', 'usaa']
    df['current_bank_group'] = np.where(df['current_bank'].isin(hdb_group), 1, 0)

    # email domain
    email_domain_group = ['gmail.com', 'yahoo.com', 'outlook.com', 'icloud.com', 'protonmail.com',
                          'ymail.com', 'me.com', 'hotmail.com', 'aol.com', 'msn.com', 'gmx.com', 'rocketmail.com', 
                          'comcast.net', 'mac.com', 'pm.me', 'mail.com', 'att.net', 'smartmomstravelagents.com', 
                          'live.com', 'proton.me', 'kw.com', 'usa.com', 'exprealty.com', 'verizon.net', 'email.com', 
                          'zohomail.com', 'bellsouth.net', 'sbcglobal.net']
    df['email_domain'] = df['email_domain'].str.lower()    
    df['email_domain_bucket'] = np.where(df['email_domain'].isin(email_domain_group), 0, 1)
        
    # industry type
    df['industry_category_name'] = df['industry_category_name'].str.lower()
    
    df['industry_category_name_professional, scientific, and technical services'] = np.where(df['industry_category_name'
                                                ]=='professional, scientific, and technical services', 1, 0)
    df['industry_category_name_real estate rental and leasing'] = np.where(df['industry_category_name']=='real estate rental and leasing', 1, 0)
    df['industry_category_name_retail trade'] = np.where(df['industry_category_name']=='retail trade', 1, 0)
    df['industry_category_name_manufacturing'] = np.where(df['industry_category_name']=='manufacturing', 1, 0)
    df['industry_category_name_administrative and support and waste management and remediation services'] = np.where(
        df['industry_category_name']=='administrative and support and waste management and remediation services', 1, 0)

    
    ################## ALLOY ##################
    # iovation_device_type
    df['iovation_device_type'] = df['iovation_device_type'].str.lower()
    df['iovation_device_type_mac'] = np.where(df['iovation_device_type']=='mac', 1, 0)
    df['iovation_device_type_android'] = np.where(df['iovation_device_type']=='android', 1, 0)
    
    # iovation_device_timezone
    df['iovation_device_timezone'] = df['iovation_device_timezone'].str.lower()
    df['iovation_device_timezone_480'] = np.where(df['iovation_device_timezone']=='480', 1, 0)

    # carrier
    df['carrier'] = df['carrier'].str.lower()
    df['carrier_tmobile'] = np.where(df['carrier'].str.contains('t-mobile*'), 1, 0)

    # socure reason codes
    df['socure_emailrisk_reason_code_i553'] = np.where(df['socure_emailrisk_reason_code'
                                                         ].str.contains("i553", case=False, na=False), 1, 0)
    df['socure_emailrisk_reason_code_i566'] = np.where(df['socure_emailrisk_reason_code'
                                                         ].str.contains("i566", case=False, na=False), 1, 0)
    df['socure_emailrisk_reason_code_r561'] = np.where(df['socure_emailrisk_reason_code'
                                                         ].str.contains("r561", case=False, na=False), 1, 0)
    df['socure_phonerisk_reason_code_i630'] = np.where(df['socure_phonerisk_reason_code'
                                                         ].str.contains("i630", case=False, na=False), 1, 0)
    df['socure_phonerisk_reason_code_i614'] = np.where(df['socure_phonerisk_reason_code'
                                                         ].str.contains("i614", case=False, na=False), 1, 0)
    df['socure_phonerisk_reason_code_r616'] = np.where(df['socure_phonerisk_reason_code'
                                                         ].str.contains("r616", case=False, na=False), 1, 0)
    df['socure_reason_code_r207'] = np.where(df['socure_reason_code'].str.contains("r207", case=False,na=False), 1, 0)
    

    ################## SEGMENT ##################
    df['sh_sw_ratio_mean'] = df['screen_height_mean']/df['screen_width_mean']
    
    # return dataframe
    return df


def get_predictions(df):
    #load model
    nsql_model = load_models()

    
    # get derived variables
    df = convert_nulls_to_one_format(df)
    df = impute_nulls_zeros(df)
    df = derive_variables(df)

    
    # predictors
    independent_variables = model.feature_names_in_
    # get probabilities
    print(nsql_model)
    print(nsql_model.predict(df[independent_variables]))
    y_prob = nsql_model.predict_proba(df[independent_variables])
    print(y_prob)
    
    # return score
    return y_prob[0,1]

def lambda_handler(event, context):
    data = json.loads(event['body'])
    
    needed_keys = ['application_id', 'estimated_monthly_revenue', 'incoming_ach_payments', 'outgoing_ach_and_checks', 
     'check_deposit_amount', 'outgoing_wire_transfers', 'incoming_wire_transfer', 'business_type', 
     'email_domain', 'current_bank', 'industry_category_name', 'iovation_device_type', 'iovation_device_timezone', 
     'carrier', 'socure_sigma', 'socure_phonerisk', 'socure_emailrisk', 'socure_reason_code', 'socure_phonerisk_reason_code', 
     'socure_emailrisk_reason_code', 'screen_width_mean', 'screen_height_mean']
    
    for key in needed_keys:
        if key not in data.keys():
            print("Key Missing")
            return {
                'statusCode': 400,
                'body': json.dumps({
                    'error': 'All parameters needed for scoring not present',
                    'message': f"please include {needed_keys}" 
                })
            }
    
    if 'application_id' in data.keys():
        try:
            score = get_predictions(pd.DataFrame.from_dict(data, orient='index').T)
        except:
            print("Scoring Failed")
            return {
                'statusCode': 400,
                'body': json.dumps({
                    'error': "Data not in expected format",
                    'message': """Check Values sent for scoring - sample values - 
                            {"application_id":"35b9598f-1d7b-4e03-b52d-5a3c7a883ada","estimated_monthly_revenue":"$5K +",
                            "incoming_ach_payments":"$5K +","outgoing_ach_and_checks":"$5K +",
                            "check_deposit_amount":"$5K +","outgoing_wire_transfers":"$0",
                            "incoming_wire_transfer":"<$1K","business_type":"llc","email_domain":"gmail.com",
                            "current_bank":"WELLS-FARGO","industry_category_name":"Manufacturing",
                            "iovation_device_type":"IPHONE","iovation_device_timezone":"300","carrier":"AT&T",
                            "socure_sigma":0.818,"socure_phonerisk":0.583,"socure_emailrisk":0.705,
                            "socure_reason_code":"[\n  \"I610\",\n  \"I626\",\n  \"I711\",\n  \"R559\",\n  \"I632\",\n  \"I705\",\n  \"I631\",\n  \"I553\",\n  \"I611\",\n  \"I614\",\n  \"R610\",\n  \"I636\",\n  \"I630\",\n  \"I708\",\n  \"I618\",\n  \"I555\",\n  \"I707\",\n  \"I602\"\n]",
                            "socure_phonerisk_reason_code":"[\n  \"I610\",\n  \"I626\",\n  \"I632\",\n  \"I620\",\n  \"I631\",\n  \"I611\",\n  \"I614\",\n  \"I636\",\n  \"I630\",\n  \"I618\",\n  \"I602\"\n]","socure_emailrisk_reason_code":"[\n  \"R559\",\n  \"I520\",\n  \"I553\",\n  \"I555\"\n]",
                            "screen_width_mean":414.0,"screen_height_mean":776.0}"""
                })
            }
    else:
        score = 0
    
    print(f"score: {score}")
    return {
        'statusCode': 200,
        'body': json.dumps({'probability': str(score)})
    }

In [33]:
event = {
        'body': json.dumps({"application_id":"35b9598f-1d7b-4e03-b52d-5a3c7a883ada","estimated_monthly_revenue":"$5K +",
                            "incoming_ach_payments":"$5K +","outgoing_ach_and_checks":"$5K +",
                            "check_deposit_amount":"$5K +","outgoing_wire_transfers":"$0",
                            "incoming_wire_transfer":"<$1K","business_type":"llc","email_domain":"gmail.com",
                            "current_bank":"WELLS-FARGO","industry_category_name":"Manufacturing",
                            "iovation_device_type":"IPHONE","iovation_device_timezone":"300","carrier":"AT&T",
                            "socure_sigma":0.818,"socure_phonerisk":0.583,"socure_emailrisk":0.705,
                            "socure_reason_code":"[\n  \"I610\",\n  \"I626\",\n  \"I711\",\n  \"R559\",\n  \"I632\",\n  \"I705\",\n  \"I631\",\n  \"I553\",\n  \"I611\",\n  \"I614\",\n  \"R610\",\n  \"I636\",\n  \"I630\",\n  \"I708\",\n  \"I618\",\n  \"I555\",\n  \"I707\",\n  \"I602\"\n]",
                            "socure_phonerisk_reason_code":"[\n  \"I610\",\n  \"I626\",\n  \"I632\",\n  \"I620\",\n  \"I631\",\n  \"I611\",\n  \"I614\",\n  \"I636\",\n  \"I630\",\n  \"I618\",\n  \"I602\"\n]","socure_emailrisk_reason_code":"[\n  \"R559\",\n  \"I520\",\n  \"I553\",\n  \"I555\"\n]",
                            "screen_width_mean":414.0,"screen_height_mean":776.0})
    }

print(lambda_handler(event, {}))

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=4, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=3, ...)
[1]
[[0.4658401 0.5341599]]
score: 0.5341598987579346
{'statusCode': 200, 'body': '{"probability": "0.5341599"}'}


In [5]:
input_keys = json.load(open('../data/nsql_sample_input.json', 'rb'))

In [6]:
input_keys.keys()

dict_keys(['application_id', 'estimated_monthly_revenue', 'incoming_ach_payments', 'outgoing_ach_and_checks', 'check_deposit_amount', 'outgoing_wire_transfers', 'incoming_wire_transfer', 'business_type', 'email_domain', 'current_bank', 'industry_category_name', 'iovation_device_type', 'iovation_device_timezone', 'carrier', 'socure_sigma', 'socure_phonerisk', 'socure_emailrisk', 'socure_reason_code', 'socure_phonerisk_reason_code', 'socure_emailrisk_reason_code', 'screen_width_mean', 'screen_height_mean'])

In [7]:
data = json.loads(event['body'])

In [8]:
df = pd.DataFrame.from_dict(data, orient='index').T

In [9]:
df

,application_id,estimated_monthly_revenue,incoming_ach_payments,outgoing_ach_and_checks,check_deposit_amount,outgoing_wire_transfers,incoming_wire_transfer,business_type,email_domain,current_bank,...,iovation_device_timezone,carrier,socure_sigma,socure_phonerisk,socure_emailrisk,socure_reason_code,socure_phonerisk_reason_code,socure_emailrisk_reason_code,screen_width_mean,screen_height_mean
0,35b9598f-1d7b-4e03-b52d-5a3c7a883ada,$5K +,$5K +,$5K +,$5K +,$0,<$1K,llc,gmail.com,WELLS-FARGO,...,300,AT&T,0.818,0.583,0.705,"[\n ""I610"",\n ""I626"",\n ""I711"",\n ""R559"",\...","[\n ""I610"",\n ""I626"",\n ""I632"",\n ""I620"",\...","[\n ""R559"",\n ""I520"",\n ""I553"",\n ""I555""\n]",414.0,776.0


In [10]:
df = convert_nulls_to_one_format(df)
df = impute_nulls_zeros(df)
df = derive_variables(df)

In [11]:
df

,application_id,estimated_monthly_revenue,incoming_ach_payments,outgoing_ach_and_checks,check_deposit_amount,outgoing_wire_transfers,incoming_wire_transfer,business_type,email_domain,current_bank,...,iovation_device_timezone_480,carrier_tmobile,socure_emailrisk_reason_code_i553,socure_emailrisk_reason_code_i566,socure_emailrisk_reason_code_r561,socure_phonerisk_reason_code_i630,socure_phonerisk_reason_code_i614,socure_phonerisk_reason_code_r616,socure_reason_code_r207,sh_sw_ratio_mean
0,35b9598f-1d7b-4e03-b52d-5a3c7a883ada,1,1,1,1,0,0,llc,gmail.com,wells-fargo,...,0,0,1,0,0,1,1,0,0,1.874396


In [12]:
independent_variables = [
                            'estimated_monthly_revenue',
                            'incoming_ach_payments',
                            'screen_width_mean',
                            'socure_emailrisk_reason_code_i553',
                            'iovation_device_type_mac',
                            'sh_sw_ratio_mean',
                            'outgoing_ach_and_checks',
                            'business_group',
                            'socure_sigma',
                            'socure_emailrisk',
                            'iovation_device_type_android',
                            'check_deposit_amount',
                            'socure_phonerisk_reason_code_i630',
                            'socure_emailrisk_reason_code_r561',
                            'socure_emailrisk_reason_code_i566',
                            'outgoing_wire_transfers',
                            'socure_phonerisk',
                            'socure_phonerisk_reason_code_i614',
                            'socure_reason_code_r207',
                            'iovation_device_timezone_480',
                            'current_bank_group',
                            'incoming_wire_transfer',
                            'socure_phonerisk_reason_code_r616',
                            'carrier_tmobile',
                            'email_domain_bucket',
                            'industry_category_name_professional, scientific, and technical services',
                            'industry_category_name_real estate rental and leasing',
                            'industry_category_name_retail trade',
                            'industry_category_name_manufacturing',
                            'industry_category_name_administrative and support and waste management and remediation services'
                        ]

In [24]:
df[independent_variables]

,estimated_monthly_revenue,incoming_ach_payments,screen_width_mean,socure_emailrisk_reason_code_i553,iovation_device_type_mac,sh_sw_ratio_mean,outgoing_ach_and_checks,business_group,socure_sigma,socure_emailrisk,...,current_bank_group,incoming_wire_transfer,socure_phonerisk_reason_code_r616,carrier_tmobile,email_domain_bucket,"industry_category_name_professional, scientific, and technical services",industry_category_name_real estate rental and leasing,industry_category_name_retail trade,industry_category_name_manufacturing,industry_category_name_administrative and support and waste management and remediation services
0,1,1,414.0,1,0,1.874396,1,1,0.818,0.705,...,0,0,0,0,0,0,0,0,1,0


In [25]:
df[model.feature_names_in_]

,estimated_monthly_revenue,incoming_ach_payments,screen_width_mean,socure_emailrisk_reason_code_i553,iovation_device_type_mac,"industry_category_name_professional, scientific, and technical services",sh_sw_ratio_mean,outgoing_ach_and_checks,business_group,industry_category_name_real estate rental and leasing,...,socure_phonerisk_reason_code_i614,socure_reason_code_r207,iovation_device_timezone_480,industry_category_name_manufacturing,current_bank_group,incoming_wire_transfer,socure_phonerisk_reason_code_r616,carrier_tmobile,industry_category_name_administrative and support and waste management and remediation services,email_domain_bucket
0,1,1,414.0,1,0,0,1.874396,1,1,0,...,1,0,0,1,0,0,0,0,0,0


In [26]:
df_flt  = df[model.feature_names_in_]

In [27]:
model = xgboost.XGBClassifier()
# model.load_model('../models/nsql_model_v2.json')
model.load_model('./nsql_model_v2.json')
# model.save_model('./nsql_model_v2.json')

In [29]:
model.predict_proba(df_flt)

array([[0.4658401, 0.5341599]], dtype=float32)

In [16]:
model.feature_names_in_

array(['estimated_monthly_revenue', 'incoming_ach_payments',
       'screen_width_mean', 'socure_emailrisk_reason_code_i553',
       'iovation_device_type_mac',
       'industry_category_name_professional, scientific, and technical services',
       'sh_sw_ratio_mean', 'outgoing_ach_and_checks', 'business_group',
       'industry_category_name_real estate rental and leasing',
       'socure_sigma', 'socure_emailrisk',
       'industry_category_name_retail trade',
       'iovation_device_type_android', 'check_deposit_amount',
       'socure_phonerisk_reason_code_i630',
       'socure_emailrisk_reason_code_r561',
       'socure_emailrisk_reason_code_i566', 'outgoing_wire_transfers',
       'socure_phonerisk', 'socure_phonerisk_reason_code_i614',
       'socure_reason_code_r207', 'iovation_device_timezone_480',
       'industry_category_name_manufacturing', 'current_bank_group',
       'incoming_wire_transfer', 'socure_phonerisk_reason_code_r616',
       'carrier_tmobile',
       'industr

In [17]:
df_flt.columns

Index(['estimated_monthly_revenue', 'incoming_ach_payments',
       'screen_width_mean', 'socure_emailrisk_reason_code_i553',
       'iovation_device_type_mac', 'sh_sw_ratio_mean',
       'outgoing_ach_and_checks', 'business_group', 'socure_sigma',
       'socure_emailrisk', 'iovation_device_type_android',
       'check_deposit_amount', 'socure_phonerisk_reason_code_i630',
       'socure_emailrisk_reason_code_r561',
       'socure_emailrisk_reason_code_i566', 'outgoing_wire_transfers',
       'socure_phonerisk', 'socure_phonerisk_reason_code_i614',
       'socure_reason_code_r207', 'iovation_device_timezone_480',
       'current_bank_group', 'incoming_wire_transfer',
       'socure_phonerisk_reason_code_r616', 'carrier_tmobile',
       'email_domain_bucket',
       'industry_category_name_professional, scientific, and technical services',
       'industry_category_name_real estate rental and leasing',
       'industry_category_name_retail trade',
       'industry_category_name_manufact

In [18]:
[ x for x in model.feature_names_in_ if x not in df_flt.columns]

[]

In [19]:
[ x for x in df_flt.columns if x not in model.feature_names_in_]

[]